In [1]:
using Dates
print(Dates.today(), " ", Dates.Time(Dates.now()))

2019-12-04 01:27:32.245

# Optimal Decision Trees - King-Rook Dataset

## Stephen Ronkowski

In [3]:
#load needed modules
using JuMP
using CSV
using DecisionTree
using StatsBase
using DataFrames
using MLDataUtils

#note - Gurobi is not FOSS - licensing required!
#this model can be solved using any MIO solver compatible with JuMP
#see http://www.juliaopt.org/JuMP.jl/v0.20.0/installation/#Getting-Solvers-1
using Gurobi

#data path declaration
FILEDIR = "/home/sronk/Downloads/Machine_Learning_MSCA_31009/Homework/data/"

"/home/sronk/Downloads/Machine_Learning_MSCA_31009/Homework/data/"

## Data Ingestion & Processing

This dataset is available [here](https://archive.ics.uci.edu/ml/machine-learning-databases/chess/king-rook-vs-king-pawn/).  Since this dataset is entirely in text, I will first transform the text into dummy-coded variables, again preserving the $0 \leq x_i \leq 1$ requirement of the model.

In [4]:
#generate header
num_header = collect(1:37)
my_header = ["col"*string(i) for i in num_header]

#load king-rook file
csv = CSV.File(FILEDIR * "kr-vs-kp.data", header = my_header)
df = DataFrame(csv)


,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
,String,String,String,String,String,String,String,String,String,String
1,f,f,f,f,f,f,f,f,f,f
2,f,f,f,f,t,f,f,f,f,f
3,f,f,f,f,t,f,t,f,f,f
4,f,f,f,f,f,f,f,f,t,f
5,f,f,f,f,f,f,f,f,f,f
6,f,f,f,f,f,f,f,f,f,f
7,f,f,f,f,f,f,f,f,t,f
8,f,f,f,f,t,f,f,f,f,f
9,f,f,f,f,f,f,f,f,f,f


In [5]:
#review current data formatting
unique(df[1,:])


5-element Array{String,1}:
 "f"  
 "l"  
 "n"  
 "t"  
 "won"

In [6]:
#initialize empty array
feature_array = zeros(size(df))

#iterate over dataframe, fill in new array with dummy-coded values
for row in 1:size(df)[1]
    for col in 1:size(df)[2]   
        if df[row,col] == "f"
            feature_array[row,col] = 0.2
        elseif df[row,col] == "l"
            feature_array[row,col] = 0.4
        elseif df[row,col] == "n"
            feature_array[row,col] = 0.6
        elseif df[row,col] == "t"
            feature_array[row,col] = 0.8
            #these two are for the y variable
        elseif df[row,col] == "won"
            feature_array[row,col] = 1
        elseif df[row,col] == "nowin"
            feature_array[row,col] = 2
        end
    end
end


In [7]:
#review loop output
feature_array


3196×37 Array{Float64,2}:
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  …  0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.8  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.8  0.2  0.8  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  …  0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.8  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.8  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.8  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.8  0.2  0.8  0.2  …  0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2     0.2  0.2  0.2  0.8  0.8  0.6  1.0
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.

In [9]:
#shuffle observations and split into train/test
feature_array = shuffleobs(feature_array, obsdim = 1)
feature_array = getobs(feature_array)
train, test = splitobs(feature_array, at = 0.7, obsdim = 1)

#extract features from input matrix, unit transform
features = train[:,1:36]

#extract labels from input matrix
labels = train[:,37]
labels = convert(Array{Int64}, labels)

#extract features from input matrix, unit transform
test_features = test[:,1:36]

#extract labels from input matrix
test_labels = test[:,37]
test_labels = convert(Array{Int64}, test_labels)


959-element Array{Int64,1}:
 1
 2
 2
 1
 2
 1
 2
 2
 1
 1
 1
 1
 2
 ⋮
 1
 1
 1
 2
 2
 1
 2
 2
 1
 2
 2
 2

The labels must be one-indexed for the model to function properly, so I confirm as much here.

In [10]:
#confirm that labels are 1-indexed
#labels = labels .- 2
unique(labels)


2-element Array{Int64,1}:
 1
 2

For sake of comparison, I will now fit a simple Decision Tree over this dataset.

In [12]:
dt_model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(dt_model, features, labels)
print_tree(dt_model, 5)

Feature 21, Threshold 0.5
L-> Feature 10, Threshold 0.5
    L-> 1 : 720/1283
    R-> 2 : 524/524
R-> 1 : 430/430


## Model Preparation

### Helper Functions


In [13]:
function findEpsilon(array)
    #capture array length for iteration
    array_len = size(array, 1)
    
    #sort array
    sorted_array = sort(array, rev = true)
    
    #initialize epsilon with arbitrarily large value
    epsilon = 1e5
    
    #iterate through consecutive values to find smallest non-zero difference
    for i in 2:(array_len - 1)
        diff = abs(sorted_array[i-1] - sorted_array[i])
        if 0 < diff < epsilon
            epsilon = diff
        end
    end
    #output smallest non-zero difference
    return epsilon
end

function epsilonArrayGenerator(matrix)
    #capture number of features in dataset, i.e. the number of columns
    matrix_rows = size(matrix, 2)
    
    #initialize output DataFrame
    epsilon_array = Vector{Float64}(undef,matrix_rows)
    
    #iterate findEpsilon function over each column
    for col in 1:matrix_rows
        epsilon_array[col] = findEpsilon(matrix[:,col])
    end
    return epsilon_array
end

function makeAncestorDict(max_nodes)
    #initialize empty dictionaries
    A_left = Dict{Int64, Vector{Int64}}()
    A_right = Dict{Int64, Vector{Int64}}()
    #A_left[1] = [1]
    #A_right[1] = [1]
    #generate keys with empty array values for each node
    for i in 1:max_nodes
        A_left[i] = []
        A_right[i] = []
    end
    #loop over all nodes, copying the left and right ancestors of the node above it
    for i in 2:max_nodes
        left_ancestors = copy(A_left[i ÷ 2])
        right_ancestors = copy(A_right[i ÷ 2])
        direct_ancestor = i ÷ 2
        A_left[i] = left_ancestors
        A_right[i] = right_ancestors
        #add a left ancestor to even nodes
        if i/2 == i ÷ 2
            append!(left_ancestors, direct_ancestor)
            A_left[i] = left_ancestors
        #add a right ancestor to odd nodes
        else
            append!(right_ancestors, direct_ancestor)
            A_right[i] = right_ancestors
        end
    end
    return A_left, A_right
end

function makeYMatrix(labels)    
    #extract dimensions for Y from label array
    num_labels = length(unique(labels))
    len_df = length(labels)
    
    #initialize empty matrix
    Y = zeros(len_df, num_labels)
    
    #set all values to -1 - this will apply a penalty to incorrect predictions
    Y = Y  .- 1
    
    #iterate n over each column, setting Y[n,k] = 1 when the label for x[i] = k
    for k in 1:num_labels
        for n in 1:len_df
            if labels[n] == k
                Y[n,k] = 1
            end
        end
    end
    return Y
end


makeYMatrix (generic function with 1 method)

### Declaration of Model Constants

In a given model space for this optimization function, there are a variety of constants that are either set by the user as a hyperparameter or generated dynamically based on those hyperparameters or the input data set.

I will begin by establishing the numerical values related to the structure of the tree, $D, N_{min}, t$, and $\alpha$  Here, $D$ is the maximum depth of the Tree, $N_{min}$ is the minimum number of samples needed to compose a leaf node, and $t$ is the total possible number of nodes in a tree.

The last of these user-declared variables, $\alpha$, is a complexity parameter that penalizes overly complex tree structures. (elaborate here)

In [14]:
#set maximum depth of tree as a constant
max_depth = 2

#minimum number of values for a given leaf node
leaf_n_min = 20

#declare complexity parameter alpha
alpha = 0.1

#find total number of nodes in tree using max_depth, t
max_nodes = 2^(max_depth+1) - 1

#initialize branch and leaf node arrays - first, find the split point between branch and leaf indices
#split point is by definition the number of nodes integer divided by two
leaf_branch_split = max_nodes ÷ 2

#total number of branches
t_b = collect(1:leaf_branch_split)

#total number of leaves
t_l = collect(leaf_branch_split+1:max_nodes)


4-element Array{Int64,1}:
 4
 5
 6
 7

Now, I will establish the model constants that are derived from the input dataset: $n, k$, and $p$.  Here, $n$ represents the number of samples within the dataset, $p$ represents the number of features in the dataset, and $k$ represents the total number of labels.

I will also establish the constant $\hat{L}$. This constant represents the "naive" prediction, namely, that every value in the dataset is a member of the most common class.  This value is then used within the optimization function (elaborate here).

In [15]:
#pull number of samples in dataset, n
num_samples = size(features, 1)

#find total number of columns in the feature space, p
num_features = size(features, 2)

#find total number of labels, k
num_labels = length(unique(labels))

#create dictionary with prediction labels as key and count of each prediction as value
output_count = countmap(labels)

#extract the count for most common label to form l_hat, which is baseline accuracy rate
l_hat = sort(collect(output_count), by = tuple -> last(tuple), rev=true)[1,1][2]/length(labels)


0.5140813589628968

For this dataset, I manually set $\epsilon_j$ to $0.01$.

In [16]:
#build ancestor dictionaries using definitions given above
A_left, A_right = makeAncestorDict(max_nodes)

#generate the epsilon array as defined earlier
#epsilon_array = epsilonArrayGenerator(features)
epsilon_array = zeros(36) .+ 0.01

#M_1 constant - defined as 1 plus the largest epsilon value
M_1 = 1 + maximum(epsilon_array)

#M constant - set equal to number of samples as rule of thumb
M = length(labels)

#generate Y matrix
Y = makeYMatrix(labels)


2237×2 Array{Float64,2}:
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
 -1.0   1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
 -1.0   1.0
  ⋮        
  1.0  -1.0
 -1.0   1.0
 -1.0   1.0
  1.0  -1.0
 -1.0   1.0
 -1.0   1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
  1.0  -1.0
 -1.0   1.0
 -1.0   1.0

### Variable and Constraint Declarations

With the constants now fixed, the model itself can now be built inside JuMP.  The first step is to declare the model itself, at which point I also limit the runtime of the optimizer to two hours.

From there, I will begin building the model by establishing the variables that model the structure of the tree itself.  The first of these are $b$, an array that captures the decision point for each node that applies a split, and $a_{j,t}$, a hot-coded matrix that indicates when feature $j$ is used to split at node $t$. Additionally, we initialize array $d$, which is hot-coded to indicate when a given branch is active (i.e. a split is applied).

In [17]:
#initialize model
model = Model(with_optimizer(Gurobi.Optimizer, Presolve=0, OutputFlag=1, TimeLimit=14400))

#b is the decision point for each branch node
#s/t a.T*x < b at a given split 
@variable(model, b[i=t_b])

#a is a hot-coded matrix that captures the variable being used to split at given branch node
@variable(model, a[j = 1:num_features, t = 1:leaf_branch_split])

#4 - establish binary constraint on a
for j in 1:num_features
    for t in t_b
        @constraint(model, a[j,t] in MOI.ZeroOne())
    end
end

#d is an indicator array equal to one when a split is applied at a given node
@variable(model, d[1:leaf_branch_split])

#constrain d to binary values
for t in t_b
    @constraint(model, d[t] in MOI.ZeroOne())
end

#2 - establish that row-wise sum of a must equal 1 for all rows - yes
for t in t_b
    #@constraint(model, sum(a[j,t] for j=1:num_features) == d[t])
    @constraint(model, sum(a[j,t] for j=1:num_features) == d[t])
    @constraint(model, sum(a[j,t] for j=1:num_features) == 1)
end

#3 - establish that b split point must be 0 <= b[i] <= d[i] - yes
for t in t_b
    @constraint(model, b[t] >= 0)
    @constraint(model, d[t] >= b[t])
end


Academic license - for non-commercial use only


Since it is possible that certain branches will not be needed to achieve an optimal solution, I will establish that only those nodes which have a split applied above them can also apply a split.  This constraint ensures that once the optimizer no longer needs to split along a given branch path (i.e. that a given branch has already achieved perfect purity) it will simply route the input features to a leaf node.

In [18]:
#5- constrain d s/t splits cannot be applied below a node that does not also split
#this does not apply to d[1] since that is the parent node and must always split
@constraint(model, d[1] == 1)

for t in 2:leaf_branch_split
    parent = t ÷ 2
    @constraint(model, d[t] <= d[parent])
end


In [19]:
#z is a hot-coded matrix captures which values are assigned to which node
@variable(model, z[i = 1:num_samples, t = t_l])

#constrain z to binary values {0,1}
for i in 1:num_samples
    for t in t_l
        @constraint(model, z[i,t] in MOI.ZeroOne())
    end
end

#l is a hot-coded array s/t l(t) = 1 when leaf node t contains any values 
@variable(model, l[t = t_l])

#contrain l to binary values {0,1}
for t in t_l
    @constraint(model, l[t] in MOI.ZeroOne())
end

#6 - constrain predictions to only be fit into nodes containing points
for i in 1:num_samples
    for t in t_l
        @constraint(model, z[i,t] <= l[t])
    end
end

#7- constrain number of samples assigned to a given leaf by lower bound 
#s/t number of samples is always greater/equal to min leaf size constant
for t in t_l
    @constraint(model, sum(z[i,t] for i in 1:num_samples) >= leaf_n_min * l[t])
end

#8 - constrain each point in data set so it can only be assigned to one leaf node
for i in 1:num_samples
    @constraint(model, sum(z[i,t] for t in t_l) == 1)
end


With the tree structure and variable spaces now established, I move on to the splitting constraints.  These constraints capture the "path" that leads to each leaf node.  For leaf node $t$, the left-hand path is taken at ancestor node(s) $A_{left}(t) = m$ when $A^T(x_i + \epsilon) \leq b(m)$, and the right-hand path is taken at ancestor node(s) $A_{right}(t) = m$ when $A^Tx_i \geq b(m)$.

(Note that equations 9-12 are intermediate steps that give the derivations below.)

In [20]:
#13 - establish left split constraints
for i in 1:num_samples  
    for t in t_l
        for m in A_left[t]
            @constraint(model, transpose(a[:,m]) * (features[i,:] + epsilon_array) <= b[m] + M_1*(1 - z[i,t]))
        end
    end
end

#14 - establish right split contraints
for i in 1:num_samples
    for t in t_l
        for m in A_right[t]      
            @constraint(model, transpose(a[:,m]) * features[i,:] >= b[m] - (1 - z[i,t]))
        end
    end
end


With the branch nodes now fully modelled, we turn to a series of variables that capture the $x_i$ features present within each node.  The first of these is $N_{k,t}$, which gives the total number of inputs with label $k$ in leaf node $t$.  Paired with this is $N_t$, which gives the sum total of inputs assigned to each leaf node.

In [21]:
#N_kt is the number of points with label k in leaf node t
@variable(model, N_kt[i = 1:num_labels, j = t_l])

#15 - establish values for N_kt[t]
for k in 1:num_labels
    for t in t_l
        @constraint(model, N_kt[k,t] == 0.5 * sum((1 + Y[i,k])*z[i,t] for i = 1:num_samples))
    end
end

#N_t is the total number of values in a leaf node t
@variable(model, N_t[i = t_l])

#16 - establish values for N_t[t] as sum of z[i,t] for each t
for t in t_l
    @constraint(model, N_t[t] == sum(z[i,t] for i = 1:num_samples))
end


Along with these variables, we capture the prediction made by each node in matrix $c_{k,t}$, which is hot-coded such that the prediction for leaf $t$ is $k$ when $c_{k,t} = 1$.

In [22]:
#c_kt is a matrix that holds the label count of each variable within a given leaf nodes
@variable(model, c_kt[i = 1:num_labels, j = t_l])

#constrain c_kt to binary values {0,1}
for k in 1:num_labels
    for t in t_l
        @constraint(model, c_kt[k,t] in MOI.ZeroOne())
    end
end

#18 - force prediction for each node with values
for t in t_l
    @constraint(model, l[t] == sum(c_kt[k,t] for k = 1:num_labels))
end


Lastly, we define loss array $L_t$, which is derived by applying a penalty factor for each prediction not in the majority class present in leaf node $t$.

In [23]:
#L is the loss for a given leaf node t
@variable(model, L[i = t_l])

#20 - set loss function lower bound
for k in 1:num_labels
    for t in t_l
        @constraint(model, L[t] >= N_t[t] - N_kt[k,t] - (M * (1 - c_kt[k,t])))
    end
end

#21 - set loss function upper bound
for k in 1:num_labels
    for t in t_l
        @constraint(model, L[t] <= N_t[t] - N_kt[k,t] + (M * c_kt[k,t]))
    end
end

#22 - set all L values to be positive
for t in t_l
   @constraint(model, L[t] >= 0) 
end

I now set the objective function, which is to minimize loss relative to the complexity of the tree as measured by the number of active branch nodes.

In [24]:
@objective(model, Min, (1/l_hat) * sum(L[t] for t in t_l)) + (alpha * sum(d[t] for t in t_b))

1.9452173913043478 L[4] + 1.9452173913043478 L[5] + 1.9452173913043478 L[6] + 1.9452173913043478 L[7] + 0.1 d[1] + 0.1 d[2] + 0.1 d[3]

With the objective function now established, we call the optimizer.

In [25]:
optimize!(model)

Academic license - for non-commercial use only
Optimize a model with 29136 rows, 9090 columns and 729505 nonzeros
Variable types: 19 continuous, 9071 integer (9071 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+03]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Variable types: 7 continuous, 9083 integer (9071 binary)
Found heuristic solution: objective 2114.4513043

Root relaxation: objective 0.000000e+00, 3983 iterations, 0.59 seconds
Total elapsed time = 6.50s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0 1754 2114.45130    0.00000   100%     -   10s
     0     0    0.00000    0 1770 2114.45130    0.00000   100%     -   11s
     0     0    0.00000    0 1768 2114.45130    0.00000   100%     -   11s
     0     0    0.00000    0 1762 2114.45130    0.00000   100%     -   16s
     0     

### Model Outputs

While the verbose output of Gurobi confirms that the optimizer ran successfully, I confirm as much below via a call to JuMP.  As expected, we see that the optimizer ended once it reached its time limit.

In [26]:
termination_status(model)

TIME_LIMIT::TerminationStatusCode = 12

I will now pull the various optimized outputs of the function.  $L_t$ gives the loss values at each leaf node.

In [27]:
value.(L)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [4, 5, 6, 7]
And data, a 4-element Array{Float64,1}:
  -0.0
 563.0
  -0.0
 156.0

$a_{j,t}$ gives the feature $j$ used to apply a split at node $t$, and $b_t$ gives the split-point value of variable $j$ at node $t$.

In [28]:
value.(a)

36×3 Array{Float64,2}:
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
  1.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
  ⋮              
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0

In [29]:
value.(b)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Array{Float64,1}:
 0.20999999999999852
 0.0                
 0.0                

$N_t$ gives the total number of $x_i$ input points assigned to leaf node $t$.

In [30]:
value.(N_t)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [4, 5, 6, 7]
And data, a 4-element Array{Float64,1}:
   -0.0
 1557.0
   -0.0
  680.0

Lastly, we see the values for $N_{k,t}$, which stores the total number of inputs with label $k$ in leaf node $t$, and $c_{k,t}$, which gives the prediction $k$ in leaf node $t$.

In [31]:
value.(N_kt)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, [4, 5, 6, 7]
And data, a 2×4 Array{Float64,2}:
  0.0  994.0   0.0  156.0
 -0.0  563.0  -0.0  524.0

In [32]:
value.(c_kt)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:2
    Dimension 2, [4, 5, 6, 7]
And data, a 2×4 Array{Float64,2}:
 -0.0   1.0  -0.0  0.0
 -0.0  -0.0  -0.0  1.0

### Measuring Model Performance

With the model now optimized, I will use the variable outputs from Gurobi via the JuMP API to determine how well the model performs.  To do so, I first establish a set of helper functions.  Firstly, I will extract the label predictions for each leaf node $t_l$, then build a function that will test whether a given $x_i$ input fits into a given leaf node, and lastly, build an aggregate function that iterates over each leaf node and attempts to fit every point into a leaf node until a match is found.  This final function will output a total accuracy percentage.

In [33]:
function labelLeafNodePredictions(N_kt)
    #extract dimensions of N_kt array
    rows = axes(N_kt)[1]
    cols = axes(N_kt)[2]
    
    #initialize empty array to store label of prediction at each leaf node
    #note that here we re-index leaf nodes such that the first leaf node is index 1
    prediction = zeros(Int8, length(cols))
    #prediction_index = cols .- (length(cols) - 1)
    #iterate over each row in N_kt, saving max value
    for c in cols
        current_max = 0
        current_prediction = 0
        for r in rows
            if value(N_kt[r,c]) > current_max
                current_max = value(N_kt[r,c])
                current_prediction = r
            end
        end
        #push best prediction to output array
        prediction[c- (length(cols) - 1)] = current_prediction
    end
    return prediction
end

function checkTreePoint(left_turns, right_turns, feature_row)
    #check if left turns are correct for leaf node
    for node in left_turns
        if transpose(value.(a[:,node])) * feature_row < value(b[node])
            continue
        else
            return false
        end
    end
    #check if right turns are correct for leaf node
    for node in right_turns
        if transpose(value.(a[:,node])) * feature_row >= value(b[node])
            continue
        else
            return false
        end            
    end
    return true
end

function predictTestPoints(test_features, test_labels)
    #ext variable
    correct_predictions = length(test_labels)
    
    #copy test labels
    lbls = copy(test_labels)
    
    #extract prediction from each leaf node
    leaf_predictions = labelLeafNodePredictions(N_kt)
    
    #iterate over leaf nodes
    for (leaf_index, leaf_value) in enumerate(t_l)
        left_turns = A_left[leaf_value]
        right_turns = A_right[leaf_value]
        #skip empty leaves
        if leaf_predictions[leaf_index] == 0
            #println("no predictions at node ", leaf_value)
            continue
        end
        for (label_index, label_value) in enumerate(lbls)
            if label_value != leaf_predictions[leaf_index]
                #println("skipping row - not matched to prediction of node")
                continue
            end
            if label_value == 0
                println("skipping row - already correctly placed")
                continue
            end       
            #println("prediction at node ", leaf_value, " is ", leaf_predictions[leaf_index])
            #println("left ancestors are ", left_turns, " ,right ancestors are ", right_turns)
            if checkTreePoint(left_turns, right_turns, test_features[label_index,:])
                lbls[label_index] = 0
                #println("found one!")
            end
        end
    end

    errors = correct_predictions - countmap(lbls)[0]
    return 1 - (errors/correct_predictions)
end
             

predictTestPoints (generic function with 1 method)

With these scoring functions in place, I can score the model's performance against the training set and the test set.

In [34]:
predictTestPoints(features, labels)

0.6785873938310236

In [37]:
predictTestPoints(test_features, test_labels)

0.694473409801877